In [1]:
import streamlit as st
import pandas as pd
import joblib

In [2]:
fichier = 'billets_production.csv'

In [3]:
# Colonnes attendues
colonnes_attendues = ['diagonal', 'height_left', 'height_right', 'margin_low', 'margin_up', 'length']

In [4]:
def verifier_colonnes(df, colonnes):
    colonnes_manquantes = [col for col in colonnes if col not in df.columns]
    if colonnes_manquantes:
        raise ValueError(f"Colonnes manquantes dans le fichier : {colonnes_manquantes}")

def verifier_valeurs_nulles(df, colonnes):
    colonnes_nulles = df[colonnes].columns[df[colonnes].isnull().any()].tolist()
    if colonnes_nulles:
        raise ValueError(f"Présence de valeurs nulles dans les colonnes : {colonnes_nulles}")

In [6]:
if fichier is not None:
    try:
        df = pd.read_csv(fichier)

        if 'id' not in df.columns:
            raise ValueError("La colonne 'id' est manquante dans le fichier CSV.")

        df = df.set_index('id')

        verifier_colonnes(df, colonnes_attendues)
        verifier_valeurs_nulles(df, colonnes_attendues)

        modele = joblib.load("prediction_billet.pkl")

        predictions = modele.predict(df)
        proba_is_genuine = modele.predict_proba(df)[:, 1]

        df['is_genuine'] = predictions
        df['proba_is_genuine'] = proba_is_genuine

    except ValueError as ve:
        print(f"❌ Erreur de validation : {ve}")

    except FileNotFoundError:
        print("❌ Le fichier du modèle 'prediction_billet.pkl' est introuvable sur le serveur.")

    except Exception as e:
        print(f"🚨 Une erreur inattendue est survenue : {e}")

else:
    print("⬆️ Veuillez charger un fichier CSV pour démarrer la prédiction.")

In [7]:
df

,diagonal,height_left,height_right,margin_low,margin_up,length,is_genuine,proba_is_genuine
id,,,,,,,,
A_1,171.76,104.01,103.54,5.21,3.30,111.42,0,0.000221
A_2,171.87,104.17,104.13,6.00,3.31,112.09,0,0.000025
A_3,172.00,104.58,104.29,4.99,3.39,111.57,0,0.000028
A_4,172.49,104.55,104.34,4.44,3.03,113.20,1,0.966538
A_5,171.65,103.63,103.56,3.77,3.16,113.33,1,0.999984
